# UAS Visi Komputer B
## Muhammad Ikram Ramadhana Friyan – 2308107010055
### Proyek: Sistem Augmented Reality Pemantau Fokus Belajar Berbasis Deteksi Wajah

Notebook ini berisi implementasi lengkap proyek UAS dengan ide **AR "Focus Timer" di Meja Belajar**. Kamera laptop digunakan untuk mendeteksi wajah pengguna, menghitung durasi fokus belajar, dan menampilkan informasi tersebut secara _augmented reality_ di atas video live.

## 1. Latar Belakang Singkat Proyek

Pada era pembelajaran daring dan tugas yang banyak, mahasiswa sering belajar di depan laptop namun sulit mengukur seberapa lama benar-benar **fokus** di depan meja. Notebook ini membangun sistem sederhana yang:

- Menggunakan **kamera laptop** untuk mendeteksi keberadaan wajah sebagai indikator fokus.
- Menghitung **durasi fokus** belajar secara otomatis.
- Menampilkan informasi tersebut secara **augmented reality (AR)** di atas video live (teks dan warna).

Proyek ini menggabungkan materi praktikum:
- Dasar Python (tipe data, waktu, perulangan) – Modul 1
- Baca video dan citra dengan OpenCV – Modul 2
- Pra-pemrosesan citra (grayscale) – Modul 3–4
- Deteksi wajah dengan Haar Cascade – Modul 7
- Konsep tracking antar frame (logika fokus vs istirahat) – Modul 8
- Augmented Reality overlay (teks di atas video) – Modul 11

## 2. Import Library dan Penjelasan

Sel ini mengimpor library yang diperlukan:
- `cv2` : OpenCV, untuk akses kamera, deteksi wajah, dan menggambar overlay.
- `time` : mengukur waktu dalam satuan detik (epoch) untuk menghitung durasi fokus.



In [1]:
import cv2
import time

print("Library cv2 (OpenCV) dan time berhasil di-import.")

Library cv2 (OpenCV) dan time berhasil di-import.


## 3. Inisialisasi Detektor Wajah dan Parameter Fokus

Sel ini menyiapkan:
- **Cascade deteksi wajah** menggunakan model Haar bawaan OpenCV.
- **VideoCapture** untuk mengakses webcam laptop (biasanya index `0`).
- Parameter dan variabel waktu:
  - `ABSENT_THRESHOLD`: berapa detik wajah hilang sebelum dianggap istirahat/distraksi.
  - `total_focus_seconds`: akumulasi total detik fokus.
  - `focus_start_time`: kapan sesi fokus aktif dimulai.
  - `last_seen_time`: kapan terakhir kali wajah terdeteksi.
  - `status`: menandai apakah saat ini `"FOKUS"` atau `"ISTIRAHAT"`.

In [3]:
# Inisialisasi detektor wajah dan parameter fokus

# Load Haar Cascade untuk deteksi wajah (modul ke-7)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

if face_cascade.empty():
    raise IOError("Gagal memuat cascade deteksi wajah. Pastikan OpenCV data haarcascades tersedia.")

# Inisialisasi akses webcam (modul ke-2)
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Tidak bisa membuka webcam. Periksa kamera dan izinnya.")

# Parameter logika fokus
ABSENT_THRESHOLD = 5.0  # detik wajah hilang sebelum dianggap istirahat

# Variabel status waktu dan fokus
total_focus_seconds = 0.0      # total waktu fokus akumulatif
focus_start_time = None        # waktu mulai sesi fokus saat ini
last_seen_time = None          # waktu terakhir wajah terlihat
status = "ISTIRAHAT"           # status awal: belum fokus

print("Detektor wajah dan webcam siap digunakan.")

Detektor wajah dan webcam siap digunakan.


## 4. Fungsi Utilitas: Format Waktu ke HH:MM:SS

Untuk menampilkan durasi fokus secara rapi, kita butuh fungsi kecil yang mengubah jumlah detik (float) menjadi string dengan format `JJ:MM:DD` (jam:menit:detik).

Sel ini mendefinisikan fungsi `format_seconds_to_hms(total_seconds)` yang:
- Membulatkan ke bawah ke detik terdekat.
- Menghitung jam, menit, dan detik.
- Mengembalikan string terformat, misalnya `"00:07:32"`.

In [4]:
def format_seconds_to_hms(total_seconds: float) -> str:
    """Mengubah detik (float) menjadi string HH:MM:SS."""
    if total_seconds < 0:
        total_seconds = 0
    total_seconds = int(total_seconds)
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

# Contoh uji fungsi
print("Contoh 452 detik:", format_seconds_to_hms(452))

Contoh 452 detik: 00:07:32


## 5. Loop Utama: Deteksi Wajah, Zona Fokus, Hitung Fokus, dan Overlay AR

Sel ini berisi **inti proyek** dengan logika yang lebih sesuai kebutuhan:

- Mengambil frame dari webcam secara terus-menerus.
- Mengubah ke grayscale dan mendeteksi wajah dengan Haar Cascade.
- Mendefinisikan **zona fokus** di tengah frame. Wajah dianggap **menghadap laptop** hanya jika pusat wajah berada di dalam zona ini.
- Mengelola logika status:
  - Jika wajah terdeteksi **dan** berada di dalam zona fokus: status `FOKUS`, mulai/tetapkan `focus_start_time`, update `last_seen_time`.
  - Jika wajah tidak berada di zona fokus (menoleh/jauh dari layar) atau tidak terdeteksi: segera akhiri sesi fokus aktif dan ubah status ke `ISTIRAHAT`.
- Menghitung total durasi fokus (`total_focus_seconds`) ditambah sesi aktif saat ini.
- Menggunakan fungsi `format_seconds_to_hms` untuk menampilkan timer fokus.
- Menggambar overlay AR di video:
  - Teks `Fokus: HH:MM:SS` di sudut kiri atas.
  - Teks status `SEDANG FOKUS` (hijau) atau `ISTIRAHAT / DISTRAKSI` (merah).
  - Kotak di sekitar wajah dan kotak besar zona fokus di tengah frame.

**Perilaku yang diinginkan:**
- Saat kamu **melihat ke laptop** (wajah di zona tengah) → timer fokus **jalan**.
- Saat kamu **tidak melihat ke laptop** (kepala menoleh jauh / keluar zona) → timer fokus **berhenti**.
- Saat kamu kembali melihat ke laptop → status kembali `FOKUS` dan timer lanjut menambah waktu.

**Cara berhenti:** tekan tombol `q` di jendela video OpenCV.

In [5]:
# Loop utama Focus Timer AR dengan zona fokus di tengah frame

print("Tekan 'q' pada jendela video untuk berhenti.")

while True:
    # 1. Baca frame dari webcam
    ret, frame = cap.read()
    if not ret:
        print("Gagal membaca frame dari kamera.")
        break

    # 2. (Opsional) perkecil ukuran untuk mempercepat proses
    scaling_factor = 0.7
    frame = cv2.resize(frame, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)

    # 3. Konversi ke grayscale untuk deteksi wajah
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 4. Hitung ukuran frame dan definisikan zona fokus di tengah
    h, w_frame = gray.shape
    # Zona fokus: kotak di tengah frame (misal 40% tengah)
    FOCUS_ZONE_X_MIN = int(0.3 * w_frame)
    FOCUS_ZONE_X_MAX = int(0.7 * w_frame)
    FOCUS_ZONE_Y_MIN = int(0.2 * h)
    FOCUS_ZONE_Y_MAX = int(0.8 * h)

    # Gambar zona fokus (bantuan visual)
    cv2.rectangle(frame,
                  (FOCUS_ZONE_X_MIN, FOCUS_ZONE_Y_MIN),
                  (FOCUS_ZONE_X_MAX, FOCUS_ZONE_Y_MAX),
                  (255, 255, 0), 2)

    # 5. Deteksi wajah dengan Haar Cascade
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    now = time.time()

    in_focus = False  # asumsi awal tidak fokus

    if len(faces) > 0:
        # Ambil wajah pertama sebagai referensi utama
        (x, y, w, h_face) = faces[0]

        # Gambar kotak di wajah (AR sederhana)
        cv2.rectangle(frame, (x, y), (x + w, y + h_face), (0, 255, 255), 2)

        # Hitung pusat wajah
        center_x = x + w // 2
        center_y = y + h_face // 2

        # Cek apakah pusat wajah berada di dalam zona fokus
        if (FOCUS_ZONE_X_MIN <= center_x <= FOCUS_ZONE_X_MAX and
                FOCUS_ZONE_Y_MIN <= center_y <= FOCUS_ZONE_Y_MAX):
            in_focus = True

    # 6. Logika status berdasarkan in_focus
    if in_focus:
        # Jika sebelumnya istirahat, mulai sesi fokus baru
        if status == "ISTIRAHAT":
            status = "FOKUS"
            focus_start_time = now
        # Update kapan terakhir wajah fokus terlihat
        last_seen_time = now
    else:
        # Jika sebelumnya fokus dan sekarang tidak fokus, akhiri sesi fokus
        if status == "FOKUS" and focus_start_time is not None:
            total_focus_seconds += now - focus_start_time
            status = "ISTIRAHAT"
            focus_start_time = None

    # 7. Hitung total detik fokus sampai saat ini
    current_focus_seconds = total_focus_seconds
    if status == "FOKUS" and focus_start_time is not None:
        current_focus_seconds += now - focus_start_time

    # 8. Konversi ke format HH:MM:SS
    focus_time_str = format_seconds_to_hms(current_focus_seconds)

    # 9. Overlay teks timer di sudut kiri atas
    cv2.putText(frame, f"Fokus: {focus_time_str}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    # 10. Overlay status di bawahnya
    if status == "FOKUS":
        status_text = "SEDANG FOKUS (di zona)"
        status_color = (0, 255, 0)  # hijau
    else:
        status_text = "ISTIRAHAT / TIDAK DI ZONA FOKUS"
        status_color = (0, 0, 255)  # merah

    cv2.putText(frame, status_text, (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, status_color, 2, cv2.LINE_AA)

    # 11. Tampilkan frame dengan overlay AR
    cv2.imshow('Focus Timer AR', frame)

    # 12. Cek tombol keluar
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# 13. Bersihkan resource
cap.release()
cv2.destroyAllWindows()

print("Total waktu fokus (detik):", total_focus_seconds)
print("Total waktu fokus (HH:MM:SS):", format_seconds_to_hms(total_focus_seconds))

Tekan 'q' pada jendela video untuk berhenti.
Total waktu fokus (detik): 3.0745530128479004
Total waktu fokus (HH:MM:SS): 00:00:03


a